# SciExt Major Work: All code

In [1]:
# importing libraries
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import pathlib
import PyQt5
import mne
import mne_bids
mne.set_log_level('warning')

from autoreject import AutoReject
import json
import scipy.io
import pandas as pd

In [2]:
cd /Volumes/big_boi/scix/ds004515

/Volumes/big_boi/scix/ds004515


## Scientific Research Proposal

### Part C: Data Collection and Cleansing Methodology

#### 1. Data Collection
Filtering is the process of reducing the power of the signal at frequencies above and below range of experimental interest with the aim of reducing noise with minimal impact on the signal. This occurs during both data collection and cleansing.

The filtering that occurs during data collection, EEG amplifier will have a low pass filter cutoff i.e. filter that cuts off frequencies above a certain threshold
Rationale: aliasing, a phenomenon where a high-frequency signal is sampled at a rate lower than the frequency of the signal, hence an artifact is produced at a lower frequency than the true high frequency source.

E.g. electromagnetic noise at 1400Hz, which the EEG amplifier (with a low pass filter cutoff set at, say, 500Hz) cannot detect. 2.8 oscillations would occur between each sample. 1400 not a multiple of 500, hence each sample captures a different phase of the 1400Hz oscillation, which put together look like a lower frequency oscillation. 

Therefore the highest frequency that can be accurately recorded at a given sampling rate is the Nyquist frequency, defined as ⅓ the sampling rate.
Online filtering = EEG hardware automatically low-pass filters data as it is recorded at threshold determined by hardware engineers to prevent aliasing.


**Importing the data**  
The data is from a system sold by Brain Products which uses a software called Brain Vision. Each dataset has three files:
1. .eeg is the actual EEG data - electrical potential measurements for all electrodes at all time points - stored in compressed (binary) format
2. .vmrk is a text file containing markers (trigger codes) which encode the onset of various stimuli, times of responses by the participant, and what the stimuli/responses were.
3. .vhdr is a text file with metadata - technical details include sampling rate, settings of EEG amplifier during recording.  

Importing the data entails representing the raw EEG data as an MNE Raw object in Python.

In [3]:
data_dir = "sub-001/eeg/"
p_file = "sub-001/eeg/sub-001_task-ProbabilisticSelection_eeg.set"
p_id = "sub-001"

In [4]:
raw_file = p_file
raw = mne.io.read_raw_eeglab(raw_file, eog=("Below Eye", "Above Eye"), preload=True) # .set file is passed

/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/2104176227.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(raw_file, eog=("Below Eye", "Above Eye"), preload=True) # .set file is passed
/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/2104176227.py:2: RuntimeWarning: Not setting positions of 2 eog channels found in montage:
['Below Eye', 'Above Eye']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_eeglab(raw_file, eog=("Below Eye", "Above Eye"), preload=True) # .set file is passed


**Exploring the data**  
Viewing the basic information and attributes of the raw dataset indicate that:
- there are 63 channels (each with data from one electrode)
- sampling rate is 500Hz (EEG data was sampled 500 times per second, or once every 2ms)
- during data collection, data was filtered between 0.02-250Hz.
- duration of data collection was 24min 28sec

In [5]:
raw.info # outputs basic information about dataset from metadata file

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,64 points
Good channels,"64 EEG, 2 EOG"
Bad channels,None
EOG channels,"Below Eye, Above Eye"
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


In [6]:
raw.describe()

<RawEEGLAB | sub-001_task-ProbabilisticSelection_eeg.fdt, 66 x 812450 (1624.9 s), ~409.2 MB, data loaded>
ch  name           type  unit        min         Q1     median         Q3        max
 0  Fp1            EEG   µV     -9804.15   -8422.35   -6821.58   -5065.97   -4138.48
 1  Fz             EEG   µV       234.62     690.82    1449.71    2359.91    3122.80
 2  F3             EEG   µV     -5351.95   -4043.64   -2515.58    -903.17     372.66
 3  F7             EEG   µV    -15589.50  -14446.88  -13120.85  -12000.31  -10314.01
 4  Below Eye      EOG   µV    -15176.90  -14290.58  -13631.35  -13194.92   -9724.90
 5  FC5            EEG   µV    -10571.92  -10006.01   -9495.21   -8543.90   -7459.91
 6  FC1            EEG   µV      3840.97    4120.90    4444.19    4577.78    4825.83
 7  C3             EEG   µV    -11807.03  -11208.69  -10368.85   -9292.53   -8008.79
 8  T7             EEG   µV     -2740.72   -1396.88    -138.23     917.87    2184.67
 9  Left Mastoid   EEG   µV    -16186.23  -1

Set channel types:

In [7]:
raw.set_channel_types(
    {
        'Empty':'misc',
        'EKG':'ecg',
        'AudioOutput':'misc'
    }
)

/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/1586516277.py:1: RuntimeWarning: The unit for channel(s) AudioOutput, Empty has changed from V to NA.
  raw.set_channel_types(


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,64 points
Good channels,"61 EEG, 2 EOG, 2 misc, 1 ECG"
Bad channels,None
EOG channels,"Below Eye, Above Eye"
ECG channels,EKG
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


Change names of left and right mastoids back to 'TP9' and 'TP10'

In [8]:
mne.rename_channels(raw.info, {
  'Left Mastoid':'TP9',
  'Right Mastoid':'TP10'
  })

Further information can be gleaned about the dataset:
- the data type of raw data is a Numpy array
- it contains 64 EEG channels + 2 EOG channels and 812,450 time points

In [9]:
# accessing further info
print(raw.info.keys()) # lists more data attributes
print(raw.info['ch_names']) # names of each channel/electrode as per the 10-20 international system
print(raw.info['chs'][0]) # info about each channel
print(raw.info['bads'])

# dimensions of data
print(type(raw._data)) # data type of dataset
print(raw._data.shape)

# set power line frequency to 60Hz
raw.info['line_freq'] = 60
print(raw.info['line_freq'])

dict_keys(['acq_pars', 'acq_stim', 'ctf_head_t', 'description', 'dev_ctf_t', 'dig', 'experimenter', 'utc_offset', 'device_info', 'file_id', 'highpass', 'hpi_subsystem', 'kit_system_id', 'helium_info', 'line_freq', 'lowpass', 'meas_date', 'meas_id', 'proj_id', 'proj_name', 'subject_info', 'xplotter_layout', 'gantry_angle', 'bads', 'chs', 'comps', 'events', 'hpi_meas', 'hpi_results', 'projs', 'proc_history', 'custom_ref_applied', 'sfreq', 'dev_head_t', 'ch_names', 'nchan'])
['Fp1', 'Fz', 'F3', 'F7', 'Below Eye', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'Above Eye', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8', 'Empty', 'EKG', 'AudioOutput']
{'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

#### 2. Cleansing/pre-processing

##### (a) Importing channel location information

In [10]:
## raw.plot_sensors(show_names=True)

A montage is the set of coordinates for each channel. In this case, this needs to be set to the 10-20 international system.
This enables us to plot the locations of channels on the scalp in a 3D Cartesian coordinate system, which is useful for visualising EEG data.

In [11]:
montage = mne.channels.make_standard_montage('standard_1020')
kept_channels = ['Fp1', 'Fz', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz',
                 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'CP6', 'CP2', 'Cz', 'C4', 'T8',
                 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7',
                 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz',
                 'PO4', 'PO8', 'P6', 'P2', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2',
                 'AF4', 'AF8', 'TP9', 'TP10']
ind = [i for (i, channel) in enumerate(montage.ch_names) if channel in kept_channels]
montage_new = montage.copy()
montage_new.ch_names = [montage.ch_names[x] for x in ind]
kept_channel_info = [montage.dig[x+3] for x in ind]
montage_new.dig = montage.dig[0:3]+kept_channel_info
## montage_new.plot();

# load montage
raw.set_montage(montage_new, on_missing='ignore')
## fig = raw.plot_sensors(show_names=True);

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,64 points
Good channels,"61 EEG, 2 EOG, 2 misc, 1 ECG"
Bad channels,None
EOG channels,"Below Eye, Above Eye"
ECG channels,EKG
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


This location information was stored in the 'dig' attribute of the raw data. Each electrode location is represented as a set of 3D Cartesian coordinates in the form (x,y,z) where x is left-right axis, y is posterior-anterior (back to front) and z is inferior-superior (down to up).  
The order of channel locations corresponds to the order of labels in the list of channel names. 

In [12]:
raw.info['dig']

[<DigPoint |        LPA : (-82.5, -0.0, 0.0) mm     : head frame>,
 <DigPoint |     Nasion : (0.0, 114.0, 0.0) mm      : head frame>,
 <DigPoint |        RPA : (82.5, 0.0, -0.0) mm      : head frame>,
 <DigPoint |     EEG #1 : (-30.9, 114.6, 27.9) mm   : head frame>,
 <DigPoint |     EEG #3 : (28.4, 115.3, 27.7) mm    : head frame>,
 <DigPoint |     EEG #5 : (-56.4, 99.2, 25.1) mm    : head frame>,
 <DigPoint |     EEG #7 : (-35.2, 109.1, 56.4) mm   : head frame>,
 <DigPoint |     EEG #9 : (-1.2, 113.7, 70.4) mm    : head frame>,
 <DigPoint |    EEG #11 : (34.2, 109.8, 57.1) mm    : head frame>,
 <DigPoint |    EEG #13 : (54.2, 99.8, 24.9) mm     : head frame>,
 <DigPoint |    EEG #16 : (-71.9, 73.1, 25.8) mm    : head frame>,
 <DigPoint |    EEG #17 : (-66.1, 80.2, 53.8) mm    : head frame>,
 <DigPoint |    EEG #18 : (-51.8, 86.7, 78.7) mm    : head frame>,
 <DigPoint |    EEG #19 : (-29.0, 91.4, 96.6) mm    : head frame>,
 <DigPoint |    EEG #20 : (-1.2, 93.3, 102.6) mm    : head fra

##### (b) Importing event codes (markers/triggers)
mark times of events of experimental interest. 

first output is NumPy array with three columns, one row for each event code. first column indicates time of event (units of samples), last column stores code (represented as integer) associated with this event.

In [13]:
raw.annotations[5]

OrderedDict([('onset', 100.858),
             ('duration', 0.002),
             ('description', 'S111'),
             ('orig_time', None)])

In [14]:
events, event_dict = mne.events_from_annotations(raw)
events

array([[     0,      0,     16],
       [ 23003,      0,     12],
       [ 25409,      0,     12],
       ...,
       [763587,      0,     13],
       [764937,      0,     13],
       [766377,      0,     11]])

In [15]:
event_id = {
    'Left Button': 1,
    'Between trials': 2,
    'Between trials': 3,
    'Between trials': 4,
    'Between trials': 5,
    'Right Button': 6,
    'FB: WIN PUPPY': 7,
    'FB: LOSE PUPPY': 8,
    'FB: WIN ALCOHOL': 9,
    'FB: LOSE ALCOHOL': 10,
    'After task': 11,
    'Before task': 12,
    'After task': 13,
    'Puppy image': 14,
    'Alcohol image': 15,
    'STATUS': 16
}

In [16]:
mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])

/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/3940604650.py:1: RuntimeWarning: event 2 missing from event_id will be ignored
  mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/3940604650.py:1: RuntimeWarning: event 3 missing from event_id will be ignored
  mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/3940604650.py:1: RuntimeWarning: event 4 missing from event_id will be ignored
  mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/3940604650.py:1: RuntimeWarning: event 11 missing from event_id will be ignored
  mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/3940604650.py:1: RuntimeWarning: More events than default colo

<Figure size 640x480 with 1 Axes>

##### (c) Filtering  
Rationale: EEG researchers record with a wider range of filter settings to prevent distortions to data caused by cutoffs too close to range of frequencies of interest. Online filtering is not permanent, allowing different filter settings to be applied and the results of this observed.

Band pass filter = filter cutoffs, high-pass of 0.1Hz, low-pass of 30Hz. This is the standard for ERP analysis since the high-pass cutoff needs to be lower by a factor of 10 than the lowest frequency of interest

Needs to be applied first to the raw EEG data before it is divided into epochs → long segments of data needed to accurately estimate and remove low frequencies. Need at least 20-30s of data to remove frequencies below 0.1Hz (one cycle every 10s). A typical segment of data is 1-2s, so low frequencies cannot be removed from these.

**Crop + filter data**

In [17]:
raw_cropped = raw.copy().crop(tmin=240, tmax=1400)
raw_filt = raw_cropped.copy().filter(l_freq=0.01, h_freq=20)

**Plot filtered data in time domain**

In [18]:
## raw_filt.plot(events=events, event_id=event_id, title='Filtered');

**Plot raw vs filtered data in frequency domain (power spectrum density)**

In [19]:
## fig, ax = plt.subplots(2)
## raw_cropped.plot_psd(ax=ax[0], show=False)
## raw_filt.plot_psd(ax=ax[1], show=False)
## ax[0].set_title('PSD before filtering')
## ax[0].set_xlabel('Frequency (Hz)')
## ax[1].set_title('PSD after filtering')
## ax[1].set_xlabel('Frequency (Hz)')
## fig.set_tight_layout(True)
## plt.show()

**Saving filtered data**

In [21]:
raw.save(data_dir + p_id + '-raw.fif', overwrite=True)
raw_filt.save(data_dir + p_id + '-filt-raw.fif', overwrite=True)

##### (d) Segmenting data into epochs around ERPs

**Specify ERP time frame**

In [22]:
tmin = -2.0
tmax = 6.0
baseline = (-0.2, 0)

epochs = mne.Epochs(raw_filt,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs

Number of events,520
Events,After task: 0Alcohol image: 80Before task: 0Between trials: 40FB: LOSE ALCOHOL: 48FB: LOSE PUPPY: 43FB: WIN ALCOHOL: 32FB: WIN PUPPY: 37Left Button: 131Puppy image: 80Right Button: 29STATUS: 0
Time range,-2.000 – 6.000 s
Baseline,-0.200 – 0.000 s


In [23]:
## epochs.plot()

**Selecting epochs based on experimental condition**

In [24]:
## epochs['FB: WIN PUPPY'].plot_image()

**Saving epochs**

In [26]:
epochs.save(data_dir + p_id + '-epochs-epo.fif', overwrite=True)

**Creating evoked data**

In [27]:
evoked_winpuppy = epochs['FB: WIN PUPPY'].average()
evoked_losepuppy = epochs['FB: LOSE PUPPY'].average()
evoked_winalcohol = epochs['FB: WIN ALCOHOL'].average()
evoked_losealcohol = epochs['FB: LOSE ALCOHOL'].average()

In [ ]:
## evoked_winpuppy.plot(spatial_colors=True)

In [ ]:
## evoked_winpuppy.plot_topomap(ch_type='eeg')

In [ ]:
## evoked_winpuppy.plot_joint(picks='eeg')

In [ ]:
## mne.viz.plot_compare_evokeds([evoked_winpuppy, evoked_losepuppy,
                              ## evoked_winalcohol, evoked_losealcohol], picks='eeg',
                              ## overwrite=True)

**Saving evoked data**

In [28]:
mne.write_evokeds(data_dir + p_id + '-evokeds-ave.fif',
                  evoked=[evoked_winpuppy, evoked_losepuppy,
                              evoked_winalcohol, evoked_losealcohol],
                              overwrite=True)

**Reading evoked data**

In [ ]:
## evokeds = mne.read_evokeds('sub-001/eeg/sub-001-evokeds-ave.fif')
## evokeds[0]

##### (e) Removing artifacts in EEG data

**reload data + epochs**

In [29]:
raw = mne.io.read_raw_fif(data_dir + p_id + '-raw.fif', preload=True)

In [30]:
epochs = mne.read_epochs(data_dir + p_id + '-epochs-epo.fif')

In [31]:
tmin = -2.0
tmax = 6.0
baseline = (-0.2, 0)

epochs_raw = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs_raw

Number of events,554
Events,After task: 13Alcohol image: 80Before task: 21Between trials: 40FB: LOSE ALCOHOL: 48FB: LOSE PUPPY: 43FB: WIN ALCOHOL: 32FB: WIN PUPPY: 37Left Button: 131Puppy image: 80Right Button: 29STATUS: 0
Time range,-2.000 – 6.000 s
Baseline,-0.200 – 0.000 s


In [32]:
epochs_raw.save(data_dir + p_id + '-epochs-raw-epo.fif', overwrite=True)

**Identify EOG and ECG artifacts**
filter data for ICA by adjusting high-pass cutoff to 1.0Hz

In [33]:
raw_ica = raw.copy().filter(l_freq=1.0, h_freq=20)

**segment raw data into epochs for ICA**

In [34]:
epochs_raw_selection = epochs_raw.selection
epochs_raw_selection

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  27,
        28,  29,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        43,  44,  45,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  59,  60,  61,  63,  64,  65,  67,  68,  69,  71,  72,  73,
        74,  75,  76,  77,  79,  80,  81,  83,  84,  85,  87,  88,  89,
        91,  92,  93,  94,  95,  96,  97,  99, 100, 101, 103, 104, 105,
       107, 108, 109, 111, 112, 113, 115, 116, 117, 119, 120, 121, 123,
       124, 125, 126, 127, 128, 129, 131, 132, 133, 135, 136, 137, 139,
       140, 141, 143, 144, 145, 146, 147, 148, 149, 151, 152, 153, 155,
       156, 157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       171, 172, 173, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 199,
       200, 201, 203, 204, 205, 207, 208, 209, 211, 212, 213, 21

In [35]:
events, event_id = mne.events_from_annotations(raw_ica)
events = events[epochs_raw_selection]

In [36]:
tmin=-2.0
tmax=6.0
baseline=(-0.2, 0)

epochs_ica = mne.Epochs(raw_ica,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        baseline=baseline,
                        preload=True, on_missing='ignore')

**run ICA on data**

In [37]:
epochs_ica.info

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,64 points
Good channels,"61 EEG, 2 EOG, 2 misc, 1 ECG"
Bad channels,None
EOG channels,"Below Eye, Above Eye"
ECG channels,EKG
Sampling frequency,500.00 Hz
Highpass,1.00 Hz
Lowpass,20.00 Hz


In [38]:
# ICA parameters
random_state = 42
n_components = 0.99

# fit ICA
ica = mne.preprocessing.ICA(n_components=n_components,
                            random_state = random_state)
ica.fit(epochs_ica)

/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/2371070891.py:8: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs_ica)


Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,31 iterations on epochs (2216554 samples)
ICA components,15
Available PCA components,61
Channel types,eeg
ICA components marked for exclusion,—


**visualise ICA components**

In [ ]:
## ica.plot_components(inst=epochs)

**detect ECG and EOG patterns**

In [39]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
ecg_evoked = ecg_epochs.average()
ecg_inds, ecg_scores = ica.find_bads_ecg(
    ecg_epochs, method='ctps')

eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
eog_evoked = eog_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(
    eog_epochs)

components_to_exclude = eog_inds
ica.exclude = components_to_exclude

**Plot ICA results**

plot automated artifact detection scores

In [ ]:
## ica.plot_scores(eog_scores)

plot ICA sources

In [ ]:
## ica.plot_sources(eog_evoked)

plot overlay of original and cleaned data

In [ ]:
## ica.plot_overlay(eog_evoked)

In [40]:
ica.exclude

[]

In [41]:
epochs_cleaned = ica.apply(epochs.copy())

/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/1468588309.py:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  epochs_cleaned = ica.apply(epochs.copy())


In [ ]:
## epochs_cleaned.plot(title='After ICA')

In [42]:
epochs_cleaned.save(data_dir + p_id + '-cleaned-ica.fif', overwrite=True)

/var/folders/sf/ns35g2p52qn763j7_gtvn4z00000gn/T/ipykernel_52717/386101422.py:1: RuntimeWarning: This filename (sub-001/eeg/sub-001-cleaned-ica.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_cleaned.save(data_dir + p_id + '-cleaned-ica.fif', overwrite=True)


##### (f) Averaging and re-referencing

In [43]:
evoked_winpuppy = epochs_cleaned['FB: WIN PUPPY'].average()
evoked_losepuppy = epochs_cleaned['FB: LOSE PUPPY'].average()
evoked_winalcohol = epochs_cleaned['FB: WIN ALCOHOL'].average()
evoked_losealcohol = epochs_cleaned['FB: LOSE ALCOHOL'].average()

In [ ]:
## evoked_winpuppy.plot(spatial_colors=True)

In [ ]:
## evoked_winpuppy.plot_topomap(ch_type='eeg')

In [ ]:
## evoked_winpuppy.plot_joint(picks='eeg')

In [ ]:
# define the channels we want plots for
## channels = ['Fz', 'Cz', 'Pz', 'Oz']

# create a figure with 4 subplots
## fig, axes = plt.subplots(2, 2, figsize=(8, 8))

# plot each channel in a separate subplot
## for idx, chan in enumerate(channels):
    ## mne.viz.plot_compare_evokeds(evokeds, 
                                ## picks=chan,
                                ## ylim={'eeg':(-10, 10)},
                                ## show_sensors='lower right',
                                ## legend='upper center',
                                ## axes=axes.reshape(-1)[idx],
                                ## show=False
                                ## );
## plt.show()    

**Computing average reference**

In [44]:
evokeds_avgref_Wpuppy = evoked_winpuppy.copy().set_eeg_reference(ref_channels='average')
evokeds_avgref_Lpuppy = evoked_losepuppy.copy().set_eeg_reference(ref_channels='average')
evokeds_avgref_Walcohol = evoked_winalcohol.copy().set_eeg_reference(ref_channels='average')
evokeds_avgref_Lalcohol = evoked_losealcohol.copy().set_eeg_reference(ref_channels='average')

In [ ]:
## mne.viz.plot_compare_evokeds([evoked_winpuppy, evoked_losepuppy,
                              ## evoked_winalcohol, evoked_losealcohol], picks='eeg');

**Re-reference data to averaged mastoids**

In [45]:
evokeds_mastoidref_Wpuppy = evokeds_avgref_Wpuppy.copy().set_eeg_reference(ref_channels=['TP9', 'TP10'])
evokeds_mastoidref_Lpuppy = evokeds_avgref_Lpuppy.copy().set_eeg_reference(ref_channels=['TP9', 'TP10'])
evokeds_mastoidref_Walcohol = evokeds_avgref_Walcohol.copy().set_eeg_reference(ref_channels=['TP9', 'TP10'])
evokeds_mastoidref_Lalcohol = evokeds_avgref_Lalcohol.copy().set_eeg_reference(ref_channels=['TP9', 'TP10'])

**Saving evoked data**

In [46]:
mne.write_evokeds(data_dir + p_id + '-refevokeds-ave.fif',
                  evoked=[evokeds_mastoidref_Wpuppy, evokeds_mastoidref_Lpuppy,
                              evokeds_mastoidref_Walcohol, evokeds_mastoidref_Lalcohol],
                              overwrite=True)

### Part D: Visualisation of Pilot data

**Load epochs**

In [47]:
evoked_W_diff = mne.combine_evoked(
    [evokeds_mastoidref_Walcohol, evokeds_mastoidref_Wpuppy],
    weights=[1, -1] # subtraction
)

evoked_W_diff.plot(gfp=True)
## mne.viz.plot_compare_evokeds(
    ## [evokeds_mastoidref_Walcohol,
    ## evokeds_mastoidref_Wpuppy,
    ## evoked_W_diff]
## )

<Figure size 640x300 with 2 Axes>

**Calculating mean amplitude**

for WIN ALCOHOL

In [53]:
# Select all of the channels and crop to the time window
channel = ['Cz']
Walcohol_roi = evokeds_mastoidref_Walcohol.copy().pick(channel).crop(
    tmin=0.2, tmax=0.4)

# Extract mean amplitude in µV over time
mean_amp_Walcohol_roi = Walcohol_roi.data.mean(axis=1) * 1e6

# Print the mean amplitude
print(mean_amp_Walcohol_roi)

[3.54515454]


for LOSE ALCOHOL:

In [49]:
# Select all of the channels and crop to the time window
channel = ['Cz']
Lalcohol_roi = evokeds_mastoidref_Lalcohol.copy().pick(channel).crop(
    tmin=0.2, tmax=0.4)

# Extract mean amplitude in µV over time
mean_amp_Lalcohol_roi = Lalcohol_roi.data.mean(axis=1) * 1e6

# Print the mean amplitude
print(mean_amp_Lalcohol_roi)

[6.16567621]


for WIN PUPPY:

In [52]:
# Select all of the channels and crop to the time window
channel = ['Cz']
Wpuppy_roi = evokeds_mastoidref_Wpuppy.copy().pick(channel).crop(
    tmin=0.2, tmax=0.4)

# Extract mean amplitude in µV over time
mean_amp_Wpuppy_roi = Wpuppy_roi.data.mean(axis=1) * 1e6

# Print the mean amplitude
print(mean_amp_Wpuppy_roi)

[-6.81830594]


for LOSE PUPPY:

In [51]:
# Select all of the channels and crop to the time window
channel = ['Cz']
Lpuppy_roi = evokeds_mastoidref_Lpuppy.copy().pick(channel).crop(
    tmin=0.2, tmax=0.4)

# Extract mean amplitude in µV over time
mean_amp_Lpuppy_roi = Lpuppy_roi.data.mean(axis=1) * 1e6

# Print the mean amplitude
print(mean_amp_Lpuppy_roi)

[14.23866273]
